In [ ]:
import pandas as pd

data_file = "training.xlsx"
data = pd.read_excel(data_file)
data

,ID,NAME,TIME,TEXT,ROLE,HATE,TARGET,VERBAL_ABUSE,INTENTION,CONTEXT,SENTIMENT
0,0,leo77,11:08:43,façon j'veux ps d'une crasseuse comme toi,bully_support,OAG,victim,DNG,AIN,NaN,NEG
1,1,Paul72,11:08:44,Claire c'est pas le mot,bully_support,NAG,NaN,NaN,OTH,NaN,NEU
2,2,@FatimahVictime,11:08:48,Je voudrais qu'on arrête de se moquer de moi e...,victim,NAG,NaN,NaN,CNS,AIN,POS
3,3,@FatimahVictime,11:09:30,Salut Julie c'est avec toi que je voulais parler,victim,NAG,NaN,NaN,OTH,NaN,NEU
4,4,julie20,11:09:39,la vie t'es fat ima,bully,CAG,victim,OTH,ATK,OTH,NEG
...,...,...,...,...,...,...,...,...,...,...,...
1127,173,amine66,15:52:19,tu kleh ou quoi,conciliator,NAG,NaN,NaN,OTH,NaN,NEU
1128,174,Leo69,15:52:20,sale feuj,bully,OAG,victim,NCG,ATK,DFN,NEG
1129,175,Aurelie,15:52:36,juif de mort,bully_support,OAG,victim,DNG,ATK,DFN,NEG
1130,176,Guillaume,15:52:40,amine vient 1v1 devant le lycéé,bully,OAG,conciliator,THR,ATK,OTH,NEG


In [1]:
import statistics
import numpy as np
import pandas as pd

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from sklearn.metrics import classification_report

def compute_metrics(data, y_true, y_pred, probs_emotions, id_prompt, output_file):
    print(classification_report(y_true, y_pred))
    data[id_prompt] = y_pred
    data['prob_'+id_prompt] = probs_emotions
    data.to_excel(output_file, index=False)

def compute_entailment(data, transfomer, template, prompts, output_file):

    print("Loading model...")

    model = AutoModelForSequenceClassification.from_pretrained(transfomer)
    tokenizer = AutoTokenizer.from_pretrained(transfomer)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    print("Predicting data using", prompts, "...")

    text_list = data['TEXT'].tolist()
    label_list = data['SENTIMENT'].tolist()
    unique_labels = sorted(list(set(label_list)))

    for id_prompt in prompts:
        y_pred = []
        y_true = []
        probs_emotions = []
        for index, text in enumerate(text_list):
            premise = text
            probs = []
            with torch.no_grad():
                for label in unique_labels:
                    if id_prompt == "emo_name":
                        # to convert the noemmo label of Blog into the natural language name "no emotion"
                        if label == "noemo":
                            label = "no emotion"
                        prompt = label
                    if id_prompt == "expr_emo":
                        prompt = template[label][0]
                    elif id_prompt == "feels_emo":
                        prompt = template[label][1]
                    elif id_prompt == "wn_def":
                        prompt = template[label][2]

                    premise = str(premise)
                    prompt = str(prompt)

                    x = tokenizer.encode(premise, prompt, return_tensors='pt', truncation_strategy='only_first')
                    x = x.to(device)
                    logits = model(x)[0]
                    entail_contradiction_logits = logits[:, [0, 2]]
                    prob_label_is_true = entail_contradiction_logits.softmax(dim=1)[:, 1]
                    probs.append(prob_label_is_true.detach().cpu().numpy()[0])
            y_pred.append(unique_labels[np.argmax(np.array(probs))])
            probs_emotions.append(probs)
            y_true.append(label_list[index])
        print("\nModel performance with prompt:", id_prompt)
        compute_metrics(data, y_true, y_pred, probs_emotions, id_prompt, output_file)

def main():
    # Parameters
    data_file = "training.xlsx"
    #data_file = "path_to_your_data_file.tsv"
    output_file = "output_file.xlsx"
    transfomer = "microsoft/deberta-v2-xlarge-mnli"
    prompts = ["emo_name", "expr_emo", "feels_emo", "wn_def"]

    template = {
        'POS': ['This text expresses a positive sentiment',
                'This person is expressing positive emotions',
                'This person is happy or pleased'],
        'NEG': ['This text expresses a negative sentiment',
                'This person is expressing negative emotions',
                'This person is sad or angry'],
        'NEU': ['This text expresses a neutral sentiment',
                'This person is expressing neutral emotions',
                'This person is neither happy nor sad']
    }
    data = pd.read_excel(data_file)
    #data = pd.read_csv(data_file, sep="\t")

    compute_entailment(data, transfomer, template, prompts, output_file)

if __name__ == "__main__":
    main()


Loading model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

...


Model performance with prompt: expr_emo
              precision    recall  f1-score   support

         NEG       0.77      0.82      0.79       816
         NEU       0.46      0.07      0.13       240
         POS       0.12      0.34      0.17        76

    accuracy                           0.63      1132
   macro avg       0.45      0.41      0.36      1132
weighted avg       0.66      0.63      0.61      1132



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2734: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(



Model performance with prompt: feels_emo
              precision    recall  f1-score   support

         NEG       0.78      0.75      0.76       816
         NEU       0.41      0.12      0.18       240
         POS       0.10      0.37      0.16        76

    accuracy                           0.59      1132
   macro avg       0.43      0.41      0.37      1132
weighted avg       0.65      0.59      0.60      1132



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2734: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(



Model performance with prompt: wn_def
              precision    recall  f1-score   support

         NEG       0.78      0.46      0.58       816
         NEU       0.23      0.02      0.04       240
         POS       0.08      0.63      0.14        76

    accuracy                           0.38      1132
   macro avg       0.36      0.37      0.25      1132
weighted avg       0.61      0.38      0.43      1132

